<a href="https://colab.research.google.com/github/saurabhsingh1411/10_simple_project_data_science/blob/main/sentimantal%20analysis/twitter_sentimental_analysis_final_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing lib and dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [2]:
data=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')


In [3]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
data = data[['tweet','label']]
data.head()

,tweet,label
0,#fingerprint #Pregnancy Test https://goo.gl/h1...,0
1,Finally a transparant silicon case ^^ Thanks t...,0
2,We love this! Would you go? #talk #makememorie...,0
3,I'm wired I know I'm George I was made that wa...,0
4,What amazing service! Apple won't even talk to...,1


In [5]:
data = data[data.label != "Neutral"]
data['tweet'] = data['tweet'].apply(lambda x: x.lower())
data['tweet'] = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['label'] == 1].size)
print(data[ data['label'] == 0].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['tweet'].values)
X = tokenizer.texts_to_sequences(data['tweet'].values)
X = pad_sequences(X)

4052
11788


In [7]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,SpatialDropout1D,Dense

In [9]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 48, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 48, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
Y = pd.get_dummies(data['label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5306, 48) (5306, 2)
(2614, 48) (2614, 2)


In [12]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 50, batch_size=batch_size, verbose = 2,validation_data=(X_test ,Y_test))

Epoch 1/50
166/166 - 40s - loss: 0.2073 - accuracy: 0.9133 - val_loss: 0.2864 - val_accuracy: 0.8711
Epoch 2/50
166/166 - 40s - loss: 0.1753 - accuracy: 0.9289 - val_loss: 0.2762 - val_accuracy: 0.8818
Epoch 3/50
166/166 - 40s - loss: 0.1462 - accuracy: 0.9429 - val_loss: 0.2834 - val_accuracy: 0.8795
Epoch 4/50
166/166 - 40s - loss: 0.1235 - accuracy: 0.9506 - val_loss: 0.3575 - val_accuracy: 0.8730
Epoch 5/50
166/166 - 40s - loss: 0.1004 - accuracy: 0.9623 - val_loss: 0.4378 - val_accuracy: 0.8646
Epoch 6/50
166/166 - 40s - loss: 0.0890 - accuracy: 0.9681 - val_loss: 0.4199 - val_accuracy: 0.8718
Epoch 7/50
166/166 - 40s - loss: 0.0773 - accuracy: 0.9717 - val_loss: 0.4533 - val_accuracy: 0.8577
Epoch 8/50
166/166 - 40s - loss: 0.0610 - accuracy: 0.9755 - val_loss: 0.4896 - val_accuracy: 0.8542
Epoch 9/50
166/166 - 40s - loss: 0.0567 - accuracy: 0.9800 - val_loss: 0.6159 - val_accuracy: 0.8546
Epoch 10/50
166/166 - 40s - loss: 0.0439 - accuracy: 0.9838 - val_loss: 0.6214 - val_accura

In [13]:
twt=test['tweet']

In [14]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(test['tweet'].values)
X_test2 = tokenizer.texts_to_sequences(test['tweet'].values)
X_test2 = pad_sequences(X)

In [ ]:
#twt = tokenizer.texts_to_sequences(twt)

In [ ]:
#twt = pad_sequences(twt, maxlen=120, dtype='int32', value=0)

In [15]:
X_test2

array([[   0,    0,    0, ...,  136,  129,    1],
       [   0,    0,    0, ...,   17,  304,  279],
       [   0,    0,    0, ...,  146,  336, 1238],
       ...,
       [   0,    0,    0, ...,  168,   39,    4],
       [   0,    0,    0, ...,    7,  343,  465],
       [   0,    0,    0, ...,  105, 1913,   52]], dtype=int32)

In [ ]:
#prediction=np.argmax(model.predict(X_test2))

In [16]:
prediction_final=np.argmax(model.predict(X_test2), axis=-1)

In [18]:
prediction_final[1:10]

array([0, 0, 0, 1, 1, 0, 0, 0, 0])